## Summary

**Data Preparation**
Below is a representation of graduate timeslots to facilitate understanding.
<img src = "Schedule_2.png" stype = "width:200px;height:400px"/>
<img src = "time2.png" stype = "width:200px;height:400px"/>

### **Input data:**
- $I$: the set of classes.
    - $a$: the set of double-unit class (i.e. 3 units for graduate and 4 units for undergraduate classes)
    - $b$: the set of single-unit class (i.e. 1.5 units for graduate and 2 units for undergraduate classes) taken in full semester
    - $c$: the set of single-unit class taken in the first half of semester
    - $d$: the set of single-unit class taken in the second half of semester
- $J$: the set of classrooms
- $T$: the set of timeslots. Below are 7 kinds of timeslots:
    - $A$: full semester for 1.5/2 hours twice a week
    - $B$: full semester for 3/4 hours once a week
    - $C$: full semester for 1.5/2 hours once a week
    - $D$: first half for 1.5/2 hours twice a week
    - $E$: second half for 1.5/2 hours twice a week
    - $F$: first half for 3/4 hours once a week
    - $G$: second half for 3/4 hours once a week
- $S$: the set of days of week, including Monday, Tuesday, Wednesday, Thursday, Friday
- $V_{s}$: the set of timeslots on day of week $s$. eg. $V_{M}$ indicates the timeslots on Monday (includ. M and MW).
- $K$: the set of professors
- $L_{k}$: the set of classes that professor $k$ teaches
- $M$: the set of back-to-back/consecutive timeslots. Timeslots are defined as back-to-back if the interval between two timeslots is less than 30 minutes
- $N$: the number of total classes to be allocated
- $U_{ij}$: the space utilization rate for allocating class $i$ to classroom $j$, calculated by seats offered over capacity
- $z_{ij}$: whether the seats offered for class $i$ is less than or equal to the capcaity of classroom $j$
- $O$: the set of conficting timeslot pairs
- $W_U,W_Q,W_R$: the weights assigned on U,Q,R respectively (by default, they are 1,1,0.2 respectively)

**Decision variables:** <br>
Let $X_{ijt}$ denote whether class $i$ is allocated to classroom $j$ at timeslot $t$(binary) ($i \in I$, $j \in J$, $t \in T$) <br>

**Auxillary Variable:** 
- $y_{jt}$: the total number of classes allocated to classroom $j$ at time $t$ (integer)
- $w_{kt}$: the total number of classes allocated to professor $k$ at time $t$ (integer)
- $U$: the average classroom utilization rate (in %) of the schedule (contineous)
- $R$: the number of professors who are allocated at least one back-to-back class (integer) 
- $Q$: the number of professors who have to work more than two days a week (integer)
- $H_{kt_1t_2}$: whether professor $k$ teaches back to back classes at consecutive timeslots $(t_1,t_2) \in M$ (binary)
- $r_{k}$: whether professor $k$ is allocated at least one back-to-back class (binary)
- $Z_{ks}$: whether professor $k$ is allocated any class on day of week $s$ (binary)
- $q_{k}$: whether professor $k$ has to work more than two days a week (binary)

<span style="color:red">**Unless otherwise noted, every summation of $i$ is over $I$, of $j$ is over $J$ and of $t$ is over $T$.**</span> 

**Objective** Maximize the schduling score <br>

$$ \text{Maximize} \qquad W_U*U - W_Q*Q + W_R*R$$

$$ U = \frac{1}{N}\sum_{i,j,t}U_{ij}X_{ijt} \qquad \text{average utilization rate} $$
- $U_{ij}$: the space utilization rate for allocating class $i$ to classroom $j$, calculated by seats offered over capacity
- $N$: the number of total classes to be allocated
- $R$ and $Q$ will be defined in constraint 5 and 6

**Constraint 1** 

**English description:** Each class eventually only occupies one classroom and one timeslot. More specifically, if it's a double-unit class, then it takes one of the timeslot in $A/B$. If it's a single-unit class taught throughtout the full semester, then it takes one of the timeslots in $C$. If it's a single-unit taught in the first half, then it takes one of the timeslots in $D/F$. If it's a single-unit class taught in the second half, then it takes one of the timeslots in $E/G$ <br>

$$\begin{aligned}
\sum_{t,j} X_{ijt} &= 1 &&\text{For every class $i \in I$}\\
\sum_{t \in A \cup B}\sum_{j} X_{ijt} &= 1 &&\text{For every double-unit class $i \in a$.}\\
\sum_{t \in C}\sum_{j} X_{ijt} &= 1 &&\text{For every single-unit class $i \in b$ taught throughtout the full semester.}\\
\sum_{t \in D \cup F}\sum_{j} X_{ijt} &= 1 &&\text{For every single-unit class $i \in c$ taught in the first half.}\\
\sum_{t \in E \cup G}\sum_{j} X_{ijt} &= 1 &&\text{For every single-unit class $i \in d$ taught in the second half .}\\
\end{aligned}$$

**Constraint 2** 

**English description:** Each classroom can not be occupied by more than one class at the same time.

$$\begin{aligned}
\sum_{i} X_{ijt} &\le  1 &&\text{for each room $j$, timeslot $t$.}\\
y_{jt_1}+y_{jt_2}  &\le 1 && \text{For each classroom $j$ and conflicting time slots $(t_1,t_2) \in O$.} \\
y_{jt} &= \sum_{i}X_{ijt} && \text{For each classroom $j$ and time slot $t$.}
\end{aligned}$$

- $y_{jt}$: the total number of classes allocated to classroom $j$ at time $t$ (integer)

**Constraint 3** 

**English description:** Each class can not be allocated to any classrooms that fail to accommodate the number of seats offered by the class. <br>

$$ \sum_{t} X_{ijt} \le z_{ij} \qquad \text{for each class $i$, classroom $j$} $$ <br>

- $z_{ij}$: whether the seats offered for class $i$ is less than or equal to the capcaity of classroom $j$

**Constraint 4** 

**English description:** Each professor can not be allocated to more than one classroom or class at the same time.
 <br>

**Breaking down the above:**

- Each professor can't be assigned to a certain timeslot in more than one classroom or class

$$ \sum_{i\in L_k}\sum_{j\in J} X_{ijt} \le 1 \quad \text{for each professor $k$, timeslot $t$} $$ <br>

- As the timeslots are designed in a way that thay can overlap. Another constraint is added to make sure the classes taught by a certain professor won't have time conflicts. For example, M/W 8-9:30am first half semester and M 8-11am full semester overlaps. In this case, we say that slot $t_1$ and $t_2$ conflicts with each other. The set of conflicting timeslots is $O$.

$$\begin{aligned}
w_{kt_1}+w_{kt_2}  &\le 1 && \text{For each professor $k$ and conflicting time slots $(t_1,t_2) \in O$.} \\
w_{kt} &= \sum_{i \in L_k}\sum_{j}X_{ijt} && \text{For each professor $k$ and time slot $t$.}
\end{aligned}$$

- $w_{kt}$: the total number of classes allocated to professor $k$ at time $t$ (integer)
- $L_{k}$: the set of classes that professor $k$ teaches

**Constraint 5** 

**English description:** Make the number of professors who have to work more than 2 days a week as small as possible.
 <br>

**Break it down:** <br>

Timeslots like MW 8:00 - 9:30 AM and M 9:30 - 11:00 AM are consecutive, let's say the set of consecutive timeslots $(t_1, t_2)$ is $M$.

**1. For each professor, we check whether there are consecutive slots occupied by the same professor's class. The middle part of formula is the total number of classes professor $k$ has to teach at two consecutive timeslots $t_1$ and $t_2$ plus 1**. 

For easy reference, let's call this number a_1. a_1 takes 3 different values: 1,2,3. 
- If $a_1 \in{[1,2]}$, i.e. the professor doesn't teach back-to-back classes at consecutive timeslots $t_1$ and $t_2$, then $H_{kt_1t_2}$ must be 0 to satisfy the LHS.
- If $a_1=3$, i.e. the professor do teach consecutive classes at $t_1$ and $t_2$, then $H_{kt_1t_2}$ must be 1 to satisfy the RHS.

That is, $H_{kt_1t_2}$ is a binary auxillary variable indicating whether the professor teaches back-to-back classes at consecutive timeslots $t_1$ and $t_2$. $N$ is the total number of classes to be allocated by the model, which is a sufficiently large upper bound. <br>

$$ 3H_{kt_1t_2} \le \sum_{i\in L_k}\sum_{j} X_{ijt_1}+ \sum_{i\in L_k}\sum_{j} X_{ijt_2}+1 \le H_{kt_1t_2}N+2 \quad \text{for each prof $k$ and consecutive time slots $(t_1,t_2)\in M$} $$

- $M$: the set of back-to-back/consecutive timeslots. Timeslots are defined as back-to-back if the interval between two timeslots is less than 30 minutes
- $N$: the number of total classes to be allocated
- $H_{kt_1t_2}$: whether professor $k$ teaches back to back classes at consecutive timeslots $(t_1,t_2) \in M$ (binary)

**2. Then, for each professor $k$, sum over all consecutive timeslots $(t_1,t_2) \in M$ to check whether he/she is allocated at least one back-to-back class. Let's say it is represented by a binary auxillary variable $r_k$** <br>

$$ r_{k} \le \sum_{(t_1,t_2) \in M} H_{kt_1t_2} \le Nr_{k} \quad \text{for each prof $k \in K$} $$

- $r_{k}$: whether professor $k$ is allocated at least one back-to-back class (binary)

**3. Sum over all professors to get the total number of professor who are allocated at least one back-to-back class** <br>

$$ R = \sum_{k\in K} r_{k}$$

- $R$: the number of professors who are allocated at least one back-to-back class (integer) 

**Constraint 6** 

**English description:** Make the number of professors who have to work more than 2 days a week as small as possible.

**Break it down:** 
<br>

**1. For each professor $k$, check whether any class is allocated for him/her on day of week $s$. Let's say it is represetned by a binary auxillary variable $Z_{ks}$.**

The middle part of the fomula is the number of courses professor $k$ teaches on day of week $s$. For easy reference, let's call this number $a_2$, if $a_2 = 0$, then $Z_{ks}$ must be $0$ to satisfy the LHS. If $a_2 \gt 0$, then $Z_{ks}$ must be $1$ to satisfy the RHS. $N$ is the total number of classes to be allocated by the model, which is larger than any possible number of classes a professor would teach on a single day.

$$ Z_{ks}\le \sum_{i\in L_k}\sum_{j}\sum_{t\in V_{s}} X_{ijt} \le Z_{ks}N \quad \text{for each prof $k \in K$ and each day of week $s \in S$} $$

- $S$: the set of days of week, including Monday, Tuesday, Wednesday, Thursday, Friday
- $V_{s}$: the set of timeslots on day of week $s$. eg. $V_{M}$ indicates the timeslots on Monday (includ. M and MW).
- $Z_{ks}$: whether professor $k$ is allocated any class on day of week $s$ (binary)
- $N$: the number of total classes to be allocated

**2. Then, for each professor $k$, check whether he/she has to work more than two days a week. Let's say it is represented by a binary auxillary variable $q_{k}$.** 

The middle part sums $Z_{ks}$ over all days of week $s \in S$ to get how many days professor $k$ has to work per week. For easy reference, let's call this number $a_3$. If $a_3 \in{[0,1,2]}$, then $q_{k}$ must be $0$ to satisfy the LHS. If $a_3 \ge 3$, then $q_{k}$ must be $1$ to satisfy the RHS.

$$ 3q_{k} \le \sum_{s \in S} Z_{ks} \le Nq_{k}+2 \quad \text{for each prof $k \in K$} $$

- $q_{k}$: whether professor $k$ has to work more than two days a week (binary) 

**3. Finally, sum over all professors to get the number of professors who have to work more than 2 days a week,$Q$**

$$ Q = \sum_{k \in K} q_{k}$$

- $Q$: the number of professors who have to work more than two days a week (integer)